In [1]:
import xarray as xr 

ds = xr.open_zarr("INCA.zarr")
ds = ds["T2M"].isel(time=slice(0,720)).load()

In [2]:
ds2 = ds.coarsen(x=2, y=2, boundary="trim").mean().load()

In [3]:
ds4 = ds2.coarsen(x=2, y=2, boundary="trim").mean().load()

In [4]:
ds8 = ds4.coarsen(x=2, y=2, boundary="trim").mean().load()

In [5]:
ds16 = ds8.coarsen(x=2, y=2, boundary="trim").mean().load()

In [ ]:
import xarray as xr
import panel as pn
import matplotlib.pyplot as plt

pn.extension()

# Load your pyramid datasets (adjust paths)
pyramids = {
    0: ds,
    1: ds2,
    2: ds4,
    3: ds8,
    4: ds16,
}

max_zoom = max(pyramids.keys())

time_vals = pyramids[max_zoom].time.values

time_slider = pn.widgets.IntSlider(start=0, end=len(time_vals)-1, value=0, name='Time')
zoom_slider = pn.widgets.IntSlider(start=0, end=max_zoom, value=max_zoom, name='Zoom level (coarser=0)')

mpl_pane = pn.pane.Matplotlib(sizing_mode='stretch_both')

def plot_slice(zoom_level, time_idx):
    ds = pyramids[zoom_level]
    fig, ax = plt.subplots(figsize=(6,6))
    ds.isel(time=time_idx).plot.imshow(ax=ax, cmap='viridis', vmin=ds.min(), vmax=ds.max())
    ax.set_title(f"Zoom: {zoom_level}, Time: {str(ds.time.values[time_idx])}")
    plt.close(fig)
    return fig

def update(event=None):
    fig = plot_slice(zoom_slider.value, time_slider.value)
    mpl_pane.object = fig

time_slider.param.watch(update, 'value')
zoom_slider.param.watch(update, 'value')

update()  # initial plot

layout = pn.Column(zoom_slider, time_slider, mpl_pane)
layout.servable()
layout.show()


Launching server at http://localhost:39383


: 

In [ ]:
import xarray as xr
import panel as pn
import matplotlib.pyplot as plt
from datetime import datetime
pn.extension()

# Open the zarr store
ds = xr.open_zarr("s1sig0.zarr", group="AT")

# Select one chunk (example indices, adjust based on chunking)
chunk_data = ds["VV"].isel(x=slice(0,500), y=slice(0,500)).sel(time=slice("2024-01-01T00:00:00.000000000","2024-02-01T00:00:00.000000000")).load()

time_values = chunk_data.time.values

time_indices = list(range(len(time_values)))

time_slider = pn.widgets.DiscreteSlider(name='Time', options=time_indices, value=0)

timestamp_display = pn.pane.Str(str(time_values[0]))

def plot_time_slice(t_index):
    fig, ax = plt.subplots(figsize=(6,6))
    chunk_data.isel(time=t_index).plot(ax=ax, vmin=-20, vmax=0)
    ax.set_title(f"Time: {time_values[t_index]}")
    plt.close(fig)
    return fig

mpl_pane = pn.pane.Matplotlib(plot_time_slice(time_slider.value), tight=True)

def update(event):
    idx = event.new
    mpl_pane.object = plot_time_slice(idx)
    timestamp_display.object = str(time_values[idx])

time_slider.param.watch(update, 'value')

layout = pn.Column(time_slider, timestamp_display, mpl_pane)
layout.servable()
layout

In [ ]:
chunk_data.time[0]

In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_dataset("INCA_data/SPARTACUS2-YEARLY_TM_2014.nc", mask_and_scale=True).load()["TM"]

In [ ]:
ds.isel(x=slice(100,200), y=slice(200,300))